In [1]:
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

train_df = pd.read_parquet("./data/train-00000-of-00001.parquet")
len(train_df)

1000000

In [2]:
for i in range(len(train_df)):

    data = train_df.iloc[i]['translation']
    print(f"Row {i}: {data['bg'][:100]} -> {data['en'][:100]}")
    
    if i == 10:
        break

Row 0: Сериозно ли? -> Are you serious?
Row 1: Моля те... -> Please...
Row 2: Тук? -> Here?
Row 3: Кога се случи това? -> When did that happen?
Row 4: Хайде, стига! -> Dude!
Row 5: Спира ти дъха. -> This is breathtaking.
Row 6: - Защо? -> - Why?
Row 7: - Да, фрау доктор. -> - Yes, doctor.
Row 8: — Негодник! -> That's bullshit.
Row 9: Може би не трябва. -> Maybe I shouldn't.
Row 10: Излизай! -> Get out!


In [3]:
from tokenizers import Tokenizer

# 1. Load the pretrained BERT case-sensitive tokenizer
#tokenizer = Tokenizer.from_pretrained("bert-base-cased")
#tokenizer = Tokenizer.from_pretrained("xlm-roberta-base")
tokenizer = Tokenizer.from_pretrained("ldilov/llama2-bg-tokenizer")
# 2. Define and add your special tokens
special_tokens = ["[CUSTOM_TOKEN_1]", "[CUSTOM_TOKEN_2]"]
tokenizer.add_special_tokens(special_tokens)

# 3. Save the tokenizer to a file
tokenizer.save("my_bert_tokenizer.json")

print("Tokenizer with special tokens saved to my_bert_tokenizer.json")

# 4. (Optional) You can also load the tokenizer back to verify
loaded_tokenizer = Tokenizer.from_file("my_bert_tokenizer.json")

# Let's test the added special tokens
text = "This is a sentence with [CUSTOM_TOKEN_1]."
encoding = loaded_tokenizer.encode(text)

print(f"\nOriginal text: {text}")
print(f"Encoded tokens: {encoding.tokens}")
print(f"Encoded IDs: {encoding.ids}")

# Verify that the special tokens are in the vocabulary
print(f"\nVocabulary size: {loaded_tokenizer.get_vocab_size()}")
print(f"ID for [CUSTOM_TOKEN_1]: {loaded_tokenizer.token_to_id('[CUSTOM_TOKEN_1]')}")

/Users/kerekovskik/repos/jax/krk_ml_utils/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tokenizer with special tokens saved to my_bert_tokenizer.json

Original text: This is a sentence with [CUSTOM_TOKEN_1].
Encoded tokens: ['<s>', '▁', 'T', 'h', 'i', 's', '▁', 'i', 's', '▁', 'a', '▁', 's', 'e', 'n', 't', 'e', 'n', 'c', 'e', '▁', 'w', 'i', 't', 'h', '▁', '[CUSTOM_TOKEN_1]', '▁', '.']
Encoded IDs: [1, 172, 59, 79, 80, 90, 172, 80, 90, 172, 72, 172, 90, 76, 85, 91, 76, 85, 74, 76, 172, 94, 80, 91, 79, 172, 32000, 172, 22]

Vocabulary size: 32002
ID for [CUSTOM_TOKEN_1]: 32000


In [4]:
print(f"Beginning Token: {loaded_tokenizer.token_to_id("<s>")}")
print(f"End Token: {loaded_tokenizer.token_to_id("</s>")}")


Beginning Token: 1
End Token: 2


In [5]:
MAX_SAMPLES = 10_000

#train_df = train_df.sample(n=MAX_SAMPLES, random_state=42).reset_index(drop=True)

In [6]:

#train_df['english_tokens'] = train_df['translation'].apply(lambda x: tokenizer.encode(x['en']).ids)
#train_df['bulgarian_tokens'] = train_df['translation'].apply(lambda x: tokenizer.encode(x['bg']).ids)

train_df['english_tokens'] = train_df['translation'].apply(lambda x: tokenizer.encode(x['en']).ids + [2])
train_df['bulgarian_tokens'] = train_df['translation'].apply(lambda x: tokenizer.encode(x['bg']).ids + [2])

for i in range(len(train_df)):

    data = train_df.iloc[i]['translation']
    print("#" * 20)
    print(f"Row {i}: {data['bg'][:100]} -> {data['en'][:100]}")
    print(f"Tokenized: {tokenizer.encode(data['bg'][:100]).tokens} -> {tokenizer.encode(data['en'][:100]).tokens}")
    
    if i == 10:
        break


####################
Row 0: Сериозно ли? -> Are you serious?
Tokenized: ['<s>', '▁С', 'ериозно▁', 'ли', '?'] -> ['<s>', '▁', 'A', 'r', 'e', '▁', 'y', 'o', 'u', '▁', 's', 'e', 'r', 'i', 'o', 'u', 's', '?']
####################
Row 1: Моля те... -> Please...
Tokenized: ['<s>', '▁Моля▁', 'те', '.', '.', '.'] -> ['<s>', '▁', 'P', 'l', 'e', 'a', 's', 'e', '.', '.', '.']
####################
Row 2: Тук? -> Here?
Tokenized: ['<s>', '▁Тук', '?'] -> ['<s>', '▁', 'H', 'e', 'r', 'e', '?']
####################
Row 3: Кога се случи това? -> When did that happen?
Tokenized: ['<s>', '▁К', 'ог', 'а▁се▁', 'случи▁', 'това', '?'] -> ['<s>', '▁', 'W', 'h', 'e', 'n', '▁', 'd', 'i', 'd', '▁', 't', 'h', 'a', 't', '▁', 'h', 'a', 'p', 'p', 'e', 'n', '?']
####################
Row 4: Хайде, стига! -> Dude!
Tokenized: ['<s>', '▁Хай', 'де', ',', '▁стиг', 'а', '!'] -> ['<s>', '▁', 'D', 'u', 'd', 'e', '!']
####################
Row 5: Спира ти дъха. -> This is breathtaking.
Tokenized: ['<s>', '▁С', 'п', 'ира▁', 'ти▁д

In [7]:
train_df.head()

,translation,english_tokens,bulgarian_tokens
0,"{'bg': 'Сериозно ли?', 'en': 'Are you serious?'}","[1, 172, 40, 89, 76, 172, 96, 86, 92, 172, 90,...","[1, 298, 19878, 286, 39, 2]"
1,"{'bg': 'Моля те...', 'en': 'Please...'}","[1, 172, 55, 83, 76, 72, 90, 76, 22, 22, 22, 2]","[1, 23995, 352, 22, 22, 22, 2]"
2,"{'bg': 'Тук?', 'en': 'Here?'}","[1, 172, 47, 76, 89, 76, 39, 2]","[1, 3535, 39, 2]"
3,"{'bg': 'Кога се случи това?', 'en': 'When did ...","[1, 172, 62, 79, 76, 85, 172, 75, 80, 75, 172,...","[1, 354, 331, 1478, 12869, 923, 39, 2]"
4,"{'bg': 'Хайде, стига!', 'en': 'Dude!'}","[1, 172, 43, 92, 75, 76, 9, 2]","[1, 14115, 285, 20, 13607, 133, 9, 2]"


In [8]:
from krk_ml_utils import datasets

datasets.create_single_npz_from_dataframe(
    file_path="./data/train.npz",
    dataframe=train_df,
    feature_columns=["english_tokens"],
    label_columns=["bulgarian_tokens"],
    features_key="x",
    targets_key="y",
    feature_dtype="int32",
    target_dtype="int32",
    compress=True,
)


Creating .npz file at: './data/train.npz'
  - File saved successfully.
  - Features array 'x' shape: (1000000,), dtype: object
  - Targets array 'y' shape: (1000000,), dtype: object


In [9]:
nlp_dataset = datasets.NumpyDataset(
    file_path="./data/train.npz",
    features_key="x",
    labels_key="y",
    rngs=None,
    allow_pickle=True
)

In [10]:
train_loader = datasets.JaxNLPDataLoader(nlp_dataset, batch_size=16, shuffle=True)

In [11]:
# Loop through first 5 batches 

i = 0
for batch in train_loader:
    x, y = batch 
    
    print(f"X shape: {x.shape}")
    print(f"Y shape: {y.shape}")
    print(type(x))
    print(x)
    i = i + 1
    
    if i == 5:
        break
    

X shape: (16, 68)
Y shape: (16, 19)
<class 'jaxlib._jax.ArrayImpl'>
[[     1    172     48 ... 250002 250002 250002]
 [     1    172     62 ... 250002 250002 250002]
 [     1    172     21 ... 250002 250002 250002]
 ...
 [     1    172     43 ... 250002 250002 250002]
 [     1    172     58 ... 250002 250002 250002]
 [     1    172     64 ... 250002 250002 250002]]
X shape: (16, 55)
Y shape: (16, 20)
<class 'jaxlib._jax.ArrayImpl'>
[[     1    172     46     92     76     90     90    172     94     79
      72     91     22      2 250002 250002 250002 250002 250002 250002
  250002 250002 250002 250002 250002 250002 250002 250002 250002 250002
  250002 250002 250002 250002 250002 250002 250002 250002 250002 250002
  250002 250002 250002 250002 250002 250002 250002 250002 250002 250002
  250002 250002 250002 250002 250002]
 [     1    172     43     80     75    172     96     86     92    172
      90     76     76    172     91     79     72     91     39      2
  250002 250002 250002

In [12]:
# Load the test df
test_df = pd.read_parquet('./data/test-00000-of-00001.parquet')
len(test_df)

2000

In [13]:
# Load the validation
val_df = pd.read_parquet('./data/validation-00000-of-00001.parquet')
len(val_df)

2000

In [14]:
test_df['english_tokens'] = test_df['translation'].apply(lambda x: tokenizer.encode(x['en']).ids  + [2])
test_df['bulgarian_tokens'] = test_df['translation'].apply(lambda x: tokenizer.encode(x['bg']).ids  + [2])
test_df.head()

,translation,english_tokens,bulgarian_tokens
0,"{'bg': 'О, Господи, смърди.', 'en': 'Oh, God, ...","[1, 172, 54, 79, 20, 172, 46, 86, 75, 20, 172,...","[1, 412, 20, 29954, 20, 2073, 23823, 22, 2]"
1,{'bg': 'Органът следва да поеме всички текущи ...,"[1, 172, 59, 79, 76, 172, 40, 92, 91, 79, 86, ...","[1, 7893, 473, 11009, 18308, 667, 9711, 1302, ..."
2,{'bg': 'Критериите за най-добър ученик в панси...,"[1, 172, 59, 79, 76, 172, 90, 76, 83, 76, 74, ...","[1, 27508, 464, 1125, 470, 21, 11967, 11145, 3..."
3,"{'bg': 'Като Джанет Ли в ""Психо"".', 'en': '-Li...","[1, 172, 21, 51, 80, 82, 76, 172, 49, 72, 85, ...","[1, 2186, 1973, 187, 189, 793, 461, 10, 119, 3..."
4,"{'bg': 'Шибаната Моника.', 'en': 'Fucking Moni...","[1, 172, 45, 92, 74, 82, 80, 85, 78, 172, 52, ...","[1, 1866, 141, 1170, 198, 116, 1513, 203, 22, 2]"


In [15]:
val_df['english_tokens'] = val_df['translation'].apply(lambda x: tokenizer.encode(x['en']).ids  + [2])
val_df['bulgarian_tokens'] = val_df['translation'].apply(lambda x: tokenizer.encode(x['bg']).ids  + [2])
val_df.head()

,translation,english_tokens,bulgarian_tokens
0,"{'bg': '- Това си го измисли.', 'en': '-You bl...","[1, 172, 21, 64, 86, 92, 172, 73, 83, 76, 94, ...","[1, 172, 21, 746, 8650, 200, 5821, 22, 2]"
1,"{'bg': 'Ще пратя тази снимка на Джаксън.', 'en...","[1, 172, 48, 172, 72, 84, 172, 90, 86, 172, 78...","[1, 2111, 1917, 5073, 334, 9853, 708, 24947, 2..."
2,"{'bg': 'Нещо е обсебило тялото й.', 'en': 'Som...","[1, 172, 58, 86, 84, 76, 91, 79, 80, 85, 78, 1...","[1, 10363, 175, 205, 380, 2782, 3703, 142, 22, 2]"
3,"{'bg': 'Добре, да вървим у нас.', 'en': 'Let's...","[1, 172, 51, 76, 91, 15, 90, 172, 94, 72, 83, ...","[1, 15185, 20, 374, 8899, 221, 31172, 22, 2]"
4,"{'bg': 'Ще ме изпратиш ли при чичо Марио?', 'e...","[1, 172, 40, 83, 83, 96, 39, 2]","[1, 2111, 408, 3903, 495, 4439, 471, 19354, 17..."


In [16]:
datasets.create_single_npz_from_dataframe(
    file_path="./data/test.npz",
    dataframe=test_df,
    feature_columns=["english_tokens"],
    label_columns=["bulgarian_tokens"],
    features_key="x",
    targets_key="y",
    feature_dtype="int32",
    target_dtype="int32",
    compress=True,
)


Creating .npz file at: './data/test.npz'
  - File saved successfully.
  - Features array 'x' shape: (2000,), dtype: object
  - Targets array 'y' shape: (2000,), dtype: object


In [17]:
datasets.create_single_npz_from_dataframe(
    file_path="./data/val.npz",
    dataframe=val_df,
    feature_columns=["english_tokens"],
    label_columns=["bulgarian_tokens"],
    features_key="x",
    targets_key="y",
    feature_dtype="int32",
    target_dtype="int32",
    compress=True,
)


Creating .npz file at: './data/val.npz'
  - File saved successfully.
  - Features array 'x' shape: (2000,), dtype: object
  - Targets array 'y' shape: (2000,), dtype: object
